### Calculator Scheme By Loan Term

#### Method 1: Flat added value services flat for all period months

In [ ]:
def getMonthlyPayment(self, EquipmentPrice, LoanTerm, terminal_rate, warranty_yrs, insurance='Yes', maintenance='Yes', extra_warranty=0, bussiness_con='Yes'): 
    markup_price = EquipmentPrice # as it has been mark upped in the input form
    principal = markup_price
    # maintenance_fee = (markup_price * self.maintenance_ratio if markup_price > 2500 else 0) if maintenance == 'Yes' else 0 
    maintenance_fee = (markup_price * self.maintenance_ratio) if maintenance == 'Yes' else 0 

    # warranty_yrs = 1 if markup_price <= 2000 else 2 if markup_price <= 5000 else 3 if markup_price <= 10000 else 5 if markup_price <= 30000 else 10
    additional_warranty = extra_warranty
    warranty_fee = markup_price * self.warranty_rate * (warranty_yrs + additional_warranty)
    insurance_fee = markup_price * self.insurance_rate * (warranty_yrs + additional_warranty) if insurance == 'Yes' else 0
    travel_labor_cost = self.travel_labor_cost * LoanTerm
    business_con_fee = (markup_price * self.business_con_rate * (warranty_yrs + additional_warranty) if insurance == 'Yes' else 0) if bussiness_con == 'Yes' else 0
    
    
    total_added_value_services = maintenance_fee + warranty_fee + insurance_fee + business_con_fee + travel_labor_cost
    total_payment = total_added_value_services + principal
    
    
    terminal_value = markup_price * terminal_rate 
    
    
    
    monthly_interest_rate = self.monthly_interest_rate
    monthly_added_value_payment = total_added_value_services / (LoanTerm*12)
    monthlyPayment = npf.pmt(monthly_interest_rate, LoanTerm*12, -principal) + monthly_added_value_payment

    self.monthlyPayment = monthlyPayment
    
    results = {
        "principal": principal,
        "total_payment": total_payment, # principal + total_added_value_services
        'total_added_value': total_added_value_services,
        "monthlyPayment": monthlyPayment,
        "maintenance_fee": maintenance_fee,
        "warranty_fee": warranty_fee,
        "insurance_fee": insurance_fee,
        "business_con_fee": business_con_fee,
        "terminal_value": terminal_value,
        'travel_labor_cost': travel_labor_cost
    }

    return results

In [ ]:
def loan_amortization(principal, annual_rate, loan_term_years, added_value_services):
    """
    Function to calculate loan amortization schedule and return a dictionary of Plotly figures.
    
    Parameters:
    principal (float): Principal loan amount.
    annual_rate (float): Annual interest rate in percent.
    loan_term_years (int): Loan term in years.
    added_value_services (float): Total added value services to be paid.
    
    Returns:
    dict: A dictionary containing Plotly figure objects for the loan amortization schedule.
    """
    # Convert APR to a monthly interest rate
    monthly_rate = annual_rate / 12  # Convert percentage to decimal and divide by 12 for monthly rate
    total_payments = loan_term_years * 12  # Total number of monthly payments

    # Calculate fixed monthly payment
    monthly_payment = npf.pmt(rate=monthly_rate, nper=total_payments, pv=-principal)
    
    monthly_added_value_payment = added_value_services / total_payments

    # Initialize list for storing results
    ratios = []

    # Calculate interest and principal portions for each month
    for month in range(1, total_payments + 1):
        interest_payment = npf.ipmt(rate=monthly_rate, per=month, nper=total_payments, pv=-principal)
        principal_payment = npf.ppmt(rate=monthly_rate, per=month, nper=total_payments, pv=-principal)
        ratios.append({
            'Month': month,
            'Interest Payment': interest_payment,
            'Principal Payment': principal_payment,
            'Added Value Payment': monthly_added_value_payment
        })

    # Create DataFrame
    df = pd.DataFrame(ratios)

    # Determine figure size dynamically based on number of data points
    width = 600 if total_payments <= 12 else 40 * total_payments
    height = 600 if total_payments <= 12 else 700

    # Create stacked bar chart using Plotly
    fig1 = go.Figure()

    # Add interest payment bar
    fig1.add_trace(go.Bar(
        x=df['Month'],
        y=df['Interest Payment'],
        name='Interest Payment',
        marker_color='purple',
        hovertext=[f"Interest Payment: ${interest:.2f}" for interest in df['Interest Payment']],
        hoverinfo='text'
    ))

    # Add principal payment bar
    fig1.add_trace(go.Bar(
        x=df['Month'],
        y=df['Principal Payment'],
        name='Principal Payment',
        marker_color='beige',
        hovertext=[f"Month: {month}<br>Principal Payment: ${principal:.2f}" for month, principal in zip(df['Month'], df['Principal Payment'])],
        hoverinfo='text'
    ))

    # Add added value payment bar
    fig1.add_trace(go.Bar(
        x=df['Month'],
        y=df['Added Value Payment'],
        name='Added Value Payment',
        marker_color='green',
        hovertext=[f"Month: {month}<br>Added Value Payment: ${added_value:.2f}" for month, added_value in zip(df['Month'], df['Added Value Payment'])],
        hoverinfo='text'
    ))

    # Update layout for the bar chart
    fig1.update_layout(
        title='Monthly Interest, Principal, and Added Value Payments for a Fixed-Rate Loan',
        xaxis_title='Month',
        yaxis_title='Payment Amount ($)',
        barmode='stack',
        legend=dict(x=0.01, y=0.99),
        width=width,
        height=height,
        hovermode='x unified',  # This setting will show hover info for all data points at the same x-value
    )

    # Calculate total interest, principal, and added value payments
    total_interest = df['Interest Payment'].sum()
    total_principal = df['Principal Payment'].sum()
    total_added_value = df['Added Value Payment'].sum()

    # Create pie chart for total principal, interest, and added value proportion with gradient blue colors
    fig2 = go.Figure(data=[go.Pie(
        labels=['Total Interest', 'Total Principal', 'Total Added Value Services'],
        values=[round(total_interest, 2), round(total_principal, 2), round(total_added_value, 2)],
        hoverinfo='label+percent+value', 
        textinfo='label+value',
        marker=dict(
            colors=['rgba(173, 216, 230, 0.8)',  # Light pastel blue for Interest
                    'rgba(135, 206, 250, 0.9)',  # Medium pastel blue for Principal
                    'rgba(176, 224, 230, 1.0)'],  # Darker pastel blue for Added Value Services
            line=dict(color='white', width=2)  # White lines between slices
        )
    )])

    # Update layout for the pie chart
    fig2.update_layout(
        title='Principal-Interest-Added Value Services Ratio',
    )

    # Return a dictionary containing both figures
    return {
        'amortization_schedule': fig1,
        'proportion_pie_chart': fig2
    }

In [ ]:
def loan_amortization_df_only(principal, annual_rate, loan_term_years, added_value_services):
    """
    Function to calculate loan amortization schedule and return a dictionary of Plotly figures.
    
    Parameters:
    principal (float): Principal loan amount.
    annual_rate (float): Annual interest rate in percent.
    loan_term_years (int): Loan term in years.

    Returns:
    dict: A dictionary containing Plotly figure objects for the loan amortization schedule.
    """
    # Convert APR to a monthly interest rate
    monthly_rate = annual_rate / 12  # Convert percentage to decimal and divide by 12 for monthly rate
    total_payments = loan_term_years * 12  # Total number of monthly payments
    
    monthly_added_value_payment = added_value_services / total_payments

    # Calculate fixed monthly payment
    monthly_payment = npf.pmt(rate=monthly_rate, nper=total_payments, pv=-principal)

    # Initialize list for storing results
    ratios = []
    remaining_principal = principal

    # Calculate interest, principal portions, and remaining principal for each month
    for month in range(1, total_payments + 1):
        interest_payment = npf.ipmt(rate=monthly_rate, per=month, nper=total_payments, pv=-principal)
        principal_payment = npf.ppmt(rate=monthly_rate, per=month, nper=total_payments, pv=-principal)
        remaining_principal -= principal_payment  # Subtract principal payment to get remaining principal
        
        ratios.append({
            'Month': month,
            'Interest Payment': interest_payment,
            'Principal Payment': principal_payment,
            'Remaining Principal': remaining_principal,
            'Added Value Payment': monthly_added_value_payment,
            'Total Payment': principal_payment + interest_payment + monthly_added_value_payment
        })

    # Create DataFrame
    df = pd.DataFrame(ratios)
    
    return df

#### Method 2: Added Value Services are paid as different subject

In [ ]:
def getMonthlyPayment(self, EquipmentPrice, LoanTerm, terminal_rate, warranty_yrs, insurance='Yes', maintenance='Yes', extra_warranty=0, bussiness_con='Yes'): 
    markup_price = EquipmentPrice # as it has been mark upped in the input form
    principal = markup_price
    # maintenance_fee = (markup_price * self.maintenance_ratio if markup_price > 2500 else 0) if maintenance == 'Yes' else 0 
    maintenance_fee = (markup_price * self.maintenance_ratio) if maintenance == 'Yes' else 0 

    # warranty_yrs = 1 if markup_price <= 2000 else 2 if markup_price <= 5000 else 3 if markup_price <= 10000 else 5 if markup_price <= 30000 else 10
    additional_warranty = extra_warranty
    warranty_fee = markup_price * self.warranty_rate * (warranty_yrs + additional_warranty)
    insurance_fee = markup_price * self.insurance_rate * (warranty_yrs + additional_warranty) if insurance == 'Yes' else 0
    travel_labor_cost = self.travel_labor_cost * LoanTerm
    business_con_fee = (markup_price * self.business_con_rate * (warranty_yrs + additional_warranty) if insurance == 'Yes' else 0) if bussiness_con == 'Yes' else 0
    
    
    total_added_value_services = maintenance_fee + warranty_fee + insurance_fee + business_con_fee + travel_labor_cost
    total_payment = total_added_value_services + principal
    
    
    terminal_value = markup_price * terminal_rate * (warranty_yrs + additional_warranty)
    
    
    
    monthly_interest_rate = self.monthly_interest_rate
    monthly_added_value_payment = total_added_value_services / (LoanTerm*12)
    monthlyPayment = npf.pmt(monthly_interest_rate, LoanTerm*12, -principal) + monthly_added_value_payment

    self.monthlyPayment = monthlyPayment
    
    results = {
        "principal": principal,
        "total_payment": total_payment, # principal + total_added_value_services
        'total_added_value': total_added_value_services,
        "monthlyPayment": monthlyPayment,
        "maintenance_fee": maintenance_fee,
        "warranty_fee": warranty_fee,
        "insurance_fee": insurance_fee,
        "business_con_fee": business_con_fee,
        "terminal_value": terminal_value,
        'travel_labor_cost': travel_labor_cost
    }

    return results